<a href="https://colab.research.google.com/github/WillHawkins3/deepfakesondemand/blob/main/Deepfakes_on_Demand_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import HfApi
from google.colab import drive
from huggingface_hub import HfApi
import pandas as pd
import requests
import os
from datetime import datetime
import time



drive.mount('/content/drive')


# Platform API Code

## Hugging Face

In [ ]:

def search_huggingface_models(search_term='stable diffusion', task='text-to-image'): # replace search term with model names of interest (e.g. "Flux", "SD")

    # Initialize API
    api = HfApi()

    # Search models
    try:
        search_results = api.list_models(
            search=search_term,
            task=task,
            full=True
        )

        model_data = []
        for model in search_results:
            model_info = {
                'Model ID': model.id,
                'Author': model.author,
                'Downloads': model.downloads,
                'Likes': model.likes,
                'Downloads all time': model.downloads_all_time,
                'Created Date': model.created_at,
            }
            model_data.append(model_info)

        # Convert to df
        df = pd.DataFrame(model_data)

        return df

    except Exception as e:
        print(f"An error occurred during the search: {e}")
        return pd.DataFrame()  # Return empty df if error

if __name__ == "__main__":
    sd_models = search_huggingface_models('stable diffusion', 'text-to-image')

    print(sd_models)

    sd_models.to_csv('/content/drive/My Drive/insert-file-name.csv', index=False)

def list_available_tasks():
    api = HfApi()
    try:
        tasks = api.list_tasks()
        print("Available Tasks:")
        for task in tasks:
            print(task)
    except Exception as e:
        print(f"Error listing tasks: {e}")

## Civitai

### Platform Analysis

In [ ]:

drive.mount('/content/drive')

def search_civitai_models(tag='Celebrity'):

    base_url = 'https://civitai.com/api/v1/models'
    all_model_data = []
    cursor = None
    total_items = 0

    while True:
        params = {
            'tag': tag,
            'limit': 100,
            'cursor': cursor
        }

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            if 'items' in data and data['items']:
                current_items = len(data['items'])
                total_items += current_items
                print(f"Fetching items with cursor: {cursor}, {current_items} items found on this request, {total_items} items found so far.")

                for model in data['items']:
                    model_info = {
                        'Model ID': model.get('id'),
                        'Name': model.get('name'),
                        'Type': model.get('type'),
                        'Description': model.get('description', 'No description'),
                        'nsfw': model.get('nsfw'),
                        'nsfw level': model.get('nsfwLevel'),
                        'minor': model.get('minor'),
                        'tags': model.get('tags'),
                        'favorites': model.get('stats', {}).get('favoriteCount', 0),
                        'Thumbs Up': model.get('stats', {}).get('thumbsUpCount', 0),
                        'Thumbs Down': model.get('stats', {}).get('thumbsDownCount', 0),
                        'Creator': model.get('creator', {}).get('username'),
                        'Downloads': model.get('stats', {}).get('downloadCount', 0),
                        'Tipped amount': model.get('stats', {}).get('tippedAmountCount', 0),
                        'Creator Name': model.get('creator', {}).get('username', 0),
                        'Date created': model.get('modelVersions', [])[0].get('publishedAt') if model.get('modelVersions') and len(model.get('modelVersions')) > 0 else None,
                        'Model URL': f"https://civitai.com/models/{model.get('id')}"
                    }
                    all_model_data.append(model_info)

                metadata = data.get('metadata', {})
                cursor = metadata.get('nextCursor')
                if cursor is None:
                    print("No more items found or 'nextCursor' missing. Stopping.")
                    break

                time.sleep(0.2)

            else:
                print(f"No more items found or 'items' key missing with cursor: {cursor}. Stopping.")
                break

        except requests.RequestException as e:
            print(f"Error during API request: {e}")
            break

    if all_model_data:
        # Create df
        df = pd.DataFrame(all_model_data)

        # Remove duplicates based on model ID
        df = df.drop_duplicates(subset='Model ID')

        # Save to Drive
        if not df.empty:
            df.to_csv('/content/drive/My Drive/Celebrity_ALL_CIVITAI_19Jan.csv', index=False)
            print(f"\nResults saved.")

        print("\nList of Models:")
        for index, model in df.iterrows():
            print(f"{index + 1}. {model['Name']} (by {model['Creator']}) - Downloads: {model['Downloads']}")

        print(f"\nTotal unique models found: {len(df)}")

        return df
    else:
        print("No models found matching your criteria.")
        return pd.DataFrame()

if __name__ == "__main__":
    celebrity_models = search_civitai_models(
        tag='Celebrity',
    )

### Model Analysis

In [ ]:

def search_civitai_models(search_term='Flux', model_type='LORA'): # replace search term with model names of interest (e.g. "Flux", "SD")

    base_url = 'https://civitai.com/api/v1/models'
    all_model_data = []
    cursor = None
    total_items = 0

    while True:
        params = {
            'query': search_term,
            'types': model_type,
            'limit': 100,
            'cursor': cursor
        }

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()


            if 'items' in data and data['items']:
                current_items = len(data['items'])
                total_items += current_items
                print(f"Fetching items with cursor: {cursor}, {current_items} items found on this request, {total_items} items found so far.")

                for model in data['items']:
                    model_info = {
                        'Model ID': model.get('id'),
                        'Name': model.get('name'),
                        'Type': model.get('type'),
                        'Description': model.get('description', 'No description'),
                        'nsfw': model.get('nsfw'),
                        'nsfw level': model.get('nsfwLevel'),
                        'minor': model.get('minor'),
                        'favorites': model.get('stats', {}).get('favoriteCount', 0),
                        'Thumbs Up': model.get('stats', {}).get('thumbsUpCount', 0),
                        'Thumbs Down': model.get('stats', {}).get('thumbsDownCount', 0),
                        'Creator': model.get('creator', {}).get('username'),
                        'Downloads': model.get('stats', {}).get('downloadCount', 0),
                        'Tipped amount': model.get('stats', {}).get('tippedAmountCount', 0),
                        'Creator Name': model.get('creator', {}).get('username', 0),
                        'Date created': model.get('modelVersions', [])[0].get('publishedAt') if model.get('modelVersions') and len(model.get('modelVersions')) > 0 else None,
                        'Model URL': f"https://civitai.com/models/{model.get('id')}"
                    }
                    all_model_data.append(model_info)

                metadata = data.get('metadata', {})
                cursor = metadata.get('nextCursor')
                if cursor is None:
                    print("No more items found or 'nextCursor' missing. Stopping.")
                    break

                time.sleep(0.2)

            else:
                print(f"No more items found or 'items' key missing with cursor: {cursor}. Stopping.")
                break

        except requests.RequestException as e:
            print(f"Error during API request: {e}")
            break

    if all_model_data:
        # Create df
        df = pd.DataFrame(all_model_data)

        # Remove duplicates based on Model ID
        df = df.drop_duplicates(subset='Model ID')

        # Save to Drive
        if not df.empty:
            df.to_csv('/content/drive/My Drive/name-your-file.csv', index=False) # change file name to your target path
            print(f"\nResults saved.")

        # Print list of models
        print("\nList of Models:")
        for index, model in df.iterrows():
            print(f"{index + 1}. {model['Name']} (by {model['Creator']}) - Downloads: {model['Downloads']}")

        # Print total results
        print(f"\nTotal unique models found: {len(df)}")

        return df
    else:
        print("No models found matching your criteria.")
        return pd.DataFrame()

if __name__ == "__main__":
    flux_models = search_civitai_models(
        search_term='Flux',
        model_type='LORA'
    )